# Caso Práctico Unidad 3: Aprendizaje Semi-Supervisado

En este caso práctico vamos a poner en práctica las técnicas de aprendizaje semi-supervisado que hemos visto en clase. 

Para este caso se han creado unos datos sintéticos que simulan un problema de clasificación binaria con 5 características. Los datos se dividen en dos datasets:
* 'supervisado_train.csv': casi 1000 muestras de entrenamiento pero únicamente 6 de ellas están etiquetadas.
* 'supervisado_test.csv': 100 muestras etiquetadas que únicamente se usarán para evaluar el rendimiento del modelo.

¿Seremos capaces de entrenar un modelo de ML con únicamente 6 muestras etiquetadas?


3. Modelo de Auto-Aprendizaje

Por último, vamos a utilizar un modelo de auto-aprendizaje y seguir los mismos pasos que en el modelo anterior.

La clase que debemos utilizar se llama `SelfTrainingClassifier`, también dentro de la librería de `sklearn.semi_supervised`. Además de poner los targets no etiquetados a -1 en este caso tenemos que enviarle el modelo base que queremos utilizar en nuestro auto-aprendizaje, prueba tanto con regresión logística como con un Support Vector Machine.



In [27]:
# Imporación de librerías y lectura de datos
# Comprueba las distribuciones del target en cada dataset
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 1. Modelo de Regresión Logística

Entrena y evalúa un modelo normal de aprendizaje semisupervisado utilizando únicamente las 6 muestras etiquetadas disponibles en train y extrae las métricas de evaluación (accuracy, precisión, recall...) con el conjunto de test. ¿Qué te parecen los resultados?

In [30]:
 # Cargar los datos
train_data = pd.read_csv('data/semisupervisado_train.csv')
test_data = pd.read_csv('data/semisupervisado_test.csv')

In [32]:
# Separar las muestras etiquetadas y no etiquetadas
train_labeled = train_data.dropna(subset=['target'])
train_unlabeled = train_data[train_data['target'].isna()]

In [34]:
# Separar características y etiquetas
X_train_labeled = train_labeled.drop(columns=['target'])
y_train_labeled = train_labeled['target']
X_train_unlabeled = train_unlabeled.drop(columns=['target'])
X_test = test_data.drop(columns=['target'])
y_test = test_data['target']

In [36]:
# Paso 1: Regresión Logística Supervisada
logistic_model = LogisticRegression()
logistic_model.fit(X_train_labeled, y_train_labeled)

# Evaluar el modelo en el conjunto de prueba
y_pred = logistic_model.predict(X_test)
accuracy_supervised = accuracy_score(y_test, y_pred)
print(f'Precisión con supervisado: {accuracy_supervised:.2f}')

Precisión con supervisado: 0.47


### 2. Modelo de Propagación de Etiquetas

Ahora vamos a utilizar todos los datos de entrenamiento disponibles mediante un modelo de aprendizaje supervisado, concretamente el de propagación de etiquetas.

Recuerda que para ello utilizaremos el modelo `LabelPropagation` dentro de la librería de `sklearn.semi_supervised` y que para entrenarlo debemos poner el target a -1 en aquellas muestras no etiquetadas.

Evalúa de nuevo el modelo con el conjunto de test y compara los resultados con el modelo anterior.

In [39]:
predicted_labels_unlabeled = logistic_model.predict(X_train_unlabeled)
train_unlabeled['target'] = predicted_labels_unlabeled

/var/folders/7w/phbtmgpx4zgf7wtnh6ql2zlr0000gn/T/ipykernel_2240/2606035121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_unlabeled['target'] = predicted_labels_unlabeled


In [41]:
# Crear el conjunto de datos combinado
combined_train_data = pd.concat([train_labeled, train_unlabeled])
X_combined_train = combined_train_data.drop(columns=['target'])
y_combined_train = combined_train_data['target']

In [43]:
# Reentrenar con datos propagados
logistic_model.fit(X_combined_train, y_combined_train)
y_pred_combined = logistic_model.predict(X_test)
accuracy_propagation = accuracy_score(y_test, y_pred_combined)
print(f'Precisión con propagación de etiquetas: {accuracy_propagation:.2f}')

Precisión con propagación de etiquetas: 0.48



### 3. Modelo de Auto-Aprendizaje

Por último, vamos a utilizar un modelo de auto-aprendizaje y seguir los mismos pasos que en el modelo anterior.

La clase que debemos utilizar se llama `SelfTrainingClassifier`, también dentro de la librería de `sklearn.semi_supervised`. Además de poner los targets no etiquetados a -1 en este caso tenemos que enviarle el modelo base que queremos utilizar en nuestro auto-aprendizaje, prueba tanto con regresión logística como con un Support Vector Machine.

In [46]:
max_iterations = 5
current_accuracy = accuracy_propagation

for iteration in range(max_iterations):
    # Predecir las etiquetas de los datos no etiquetados
    predicted_labels_unlabeled = logistic_model.predict(X_train_unlabeled)
    train_unlabeled['target'] = predicted_labels_unlabeled

    # Recombinar los datos
    combined_train_data = pd.concat([train_labeled, train_unlabeled])
    X_combined_train = combined_train_data.drop(columns=['target'])
    y_combined_train = combined_train_data['target']

    # Reentrenar el modelo
    logistic_model.fit(X_combined_train, y_combined_train)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred_combined = logistic_model.predict(X_test)
    new_accuracy = accuracy_score(y_test, y_pred_combined)

    # Detener si no hay mejora
    if new_accuracy > current_accuracy:
        current_accuracy = new_accuracy
    else:
        break

print(f'Precisión con autoaprendizaje: {current_accuracy:.2f}')

Precisión con autoaprendizaje: 0.48


/var/folders/7w/phbtmgpx4zgf7wtnh6ql2zlr0000gn/T/ipykernel_2240/3720480238.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_unlabeled['target'] = predicted_labels_unlabeled


In [48]:
# 
# En resumen, aunque los métodos semisupervisados aplicados
# muestran una ligera mejora, con tan pocos datos etiquetados es difícil obtener un modelo confiable
#